In [1]:
# some tools:
import json
from functools import partial
from cjm_psl_utils.core import download_file, file_extract
from cjm_torchvision_tfms.core import ResizeMax, PadSquare, CustomRandomIoUCrop
import math


import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
torch.manual_seed(42)
import numpy as np
from PIL import Image
torchvision.disable_beta_transforms_warning()
from torchvision.tv_tensors import BoundingBoxes
from torchvision.utils import draw_bounding_boxes
import torchvision.transforms.v2  as transforms

import pytorch_lightning as pl
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from torch.autograd import Variable
from pytorch_lightning import loggers as pl_loggers


BASE_PATH = "/Users/sacha.lahlou/Library/CloudStorage/OneDrive-PMU/centernetImplementation/centernet/cardDetectionDataset/"
TEST_PATH = "test/"
TRAIN_PATH = "train/"
VAL_PATH = "valid/"
ANNOTATION_FILENAME = "_annotations.coco.json"

NB_CLASS = 53

TRAIN_SIZE = 256

/opt/miniconda3/envs/devIA/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CardDlayingDataset(Dataset):
    def __init__(self, path, transforms=None):
        super().__init__()
        self.path = path

        with open(self.path + '_annotations.coco.json', 'r') as f:
            datas = json.load(f)
        classes = datas['categories']
        imgs = datas['images']
        annotations = datas['annotations']
        
        self.parsed_datas = self._datas_parser(imgs, annotations)
        self.classes_names = [c['name'] for c in classes] # class_name are already in order

        
        self.size = len(self.parsed_datas)
        self.transforms = transforms

        self.max_objs = 5
        self.padding = 127  # 31 for resnet/resdcn
        self.downsampling_ratio = 4
        self.img_size = {'h': TRAIN_SIZE, 'w': TRAIN_SIZE}
        self.fmap_size = {'h': TRAIN_SIZE // self.downsampling_ratio, 'w': TRAIN_SIZE // self.downsampling_ratio}
        # self.rand_scales = np.arange(0.6, 1.4, 0.1)
        self.gaussian_iou = 0.3


        
    def __getitem__(self, i):
        img = Image.open(self.path + self.parsed_datas[i]['filename']).convert('RGB')
        labels = self.parsed_datas[i]['labels']
        
        return (img, labels)

    def __len__(self):
        return self.size
    
    def _datas_parser(self, imgs, annotations):
        images_list_parsed = []

        for img in imgs:
            img_id, filename = img['id'], img['file_name']
            related_annotations = []
            for annot in annotations:
                if annot["image_id"] == img_id:
                    related_annotations.append(dict( box=annot['bbox'], classe=annot['category_id'], area=annot['area']) )
            images_list_parsed.append(dict(id=img_id, filename=filename, labels=related_annotations))

        return (images_list_parsed)
    

In [3]:
ds = CardDlayingDataset(BASE_PATH + TRAIN_PATH)

In [4]:

img, label = ds[0]

In [5]:
mean_sum = torch.zeros(3)
var_sum = torch.zeros(3)

for i in range(ds.__len__()):
    img, label = ds[i]
    img_tensor = transforms.ToTensor()(img)
    mean_sum += torch.mean(img_tensor, dim=(1, 2))  # Compute mean along height and width dimensions
    var_sum += torch.var(img_tensor, dim=(1, 2))

mean = mean_sum / ds.__len__()
variance = var_sum / ds.__len__()

print('mean:')
print((mean))
print('var:')
print((variance))

/opt/miniconda3/envs/devIA/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


mean:
tensor([0.5522, 0.5133, 0.4826])
var:
tensor([0.0650, 0.0678, 0.0696])
